# EDA

## Month-over-month churn

This section attempts to determine the number of households that are re-interviewed in consecutive months in 2020. 

First, as a proof-of-concept, I'll calculate the number of households that were re-interviewed in Feb 2020 after having been interviewed in Jan 2020.

In [1]:
%load_ext autoreload
%autoreload 2

import os
import sys

module_path = os.path.abspath(os.path.join(os.pardir, os.pardir, 'src'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from modules import dataloading as dl
from modules import cleaning as c
targetdir = '../../src/data/'

In [2]:
# import data'
jan_2020_df = dl.CPS_raw(targetdir, ['jan2020'], None)
feb_2020_df = dl.CPS_raw(targetdir, ['jan2020'], None)
jan_feb_2020_df = dl.CPS_raw(targetdir, ['jan2020', 'feb2020'], None)


In [71]:
# clean DF
jan_2020_df = c.clean_CPS_df(jan_2020_df)
feb_2020_df = c.clean_CPS_df(feb_2020_df)
jan_feb_2020_df = c.clean_CPS_df(jan_feb_2020_df)

## Trying to uniquely identify each row

In [38]:
jan_feb_2020_df.ID.value_counts().describe()

count    88822.000000
mean         3.129236
std          2.380955
min          1.000000
25%          2.000000
50%          2.000000
75%          4.000000
max         26.000000
Name: ID, dtype: float64

It appears that there are a large number of rows with a large number of unique household IDs. We need to check out what differentiates these rows.

None of the rows in January are duplicates.

In [15]:
jan_2020_df.groupby('ID').filter(lambda x: len(x) > 1).duplicated().value_counts()

False    102952
dtype: int64

By checking the number of people in each household you can tell that HRHHID + HRHHID2 quite literally is about households (combined together they become `HH_ID`. If you include PERRP, the relationship to the reference person, however, you see that you start getting unique identifiers (`IND_ID`). 

In [42]:
jan_2020_df[(jan_2020_df.HRNUMHOU == 2)].head(6)

,HRHHID,HRMONTH,HRYEAR4,HURESPLI,HUFINAL,HETENURE,HEHOUSUT,HETELHHD,HETELAVL,HEPHONEO,...,PECERT1,PECERT2,PECERT3,PXCERT1,PXCERT2,PXCERT3,ID,ID_PERRP,HH_ID,IND_ID
0,4795110719,1,2020,1,201,-1,1,1,-1,1,...,2,-1,-1,20,0,0,47951107199011,4795110719901140,47951107199011,4795110719901140
1,4795110719,1,2020,1,201,-1,1,1,-1,1,...,2,-1,-1,20,0,0,47951107199011,4795110719901142,47951107199011,4795110719901142
8,110107755856,1,2020,1,201,-1,1,1,-1,0,...,2,-1,-1,20,0,0,11010775585611111,1101077558561111140,11010775585611111,1101077558561111140
9,110107755856,1,2020,1,201,-1,1,1,-1,0,...,2,-1,-1,20,0,0,11010775585611111,1101077558561111148,11010775585611111,1101077558561111148
17,110359424339,1,2020,1,201,-1,1,1,-1,1,...,1,1,1,20,20,20,11035942433910111,1103594243391011140,11035942433910111,1103594243391011140
18,110359424339,1,2020,1,201,-1,1,1,-1,1,...,2,-1,-1,20,0,0,11035942433910111,1103594243391011150,11035942433910111,1103594243391011150


In [35]:
jan_2020_df[(jan_2020_df.HRNUMHOU == 1)].head(6)

,HRHHID,HRMONTH,HRYEAR4,HURESPLI,HUFINAL,HETENURE,HEHOUSUT,HETELHHD,HETELAVL,HEPHONEO,...,PEPDEMP2,PTNMEMP2,PECERT1,PECERT2,PECERT3,PXCERT1,PXCERT2,PXCERT3,ID,ID_PERRP
10,110116792163,1,2020,1,201,-1,1,1,-1,1,...,-1,-1,2,-1,-1,20,0,0,11011679216311011,1101167921631101141
16,110327856469,1,2020,1,201,-1,1,1,-1,1,...,-1,-1,2,-1,-1,20,0,0,1103278564699111,110327856469911141
19,110405887199,1,2020,1,201,-1,1,1,-1,1,...,-1,-1,2,-1,-1,20,0,0,11040588719911011,1104058871991101141
22,110447170975,1,2020,1,201,-1,1,1,-1,1,...,-1,-1,2,-1,-1,0,0,0,11044717097511011,1104471709751101141
29,110509947170,1,2020,1,201,-1,1,1,-1,1,...,-1,-1,1,2,1,20,20,20,11050994717010011,1105099471701001141
30,110509947170,1,2020,1,201,-1,1,1,-1,1,...,-1,-1,2,-1,-1,20,0,0,1105099471709011,110509947170901141


In [40]:
jan_2020_df[(jan_2020_df.HRHHID == 4795110719)].head(6)

,HRHHID,HRMONTH,HRYEAR4,HURESPLI,HUFINAL,HETENURE,HEHOUSUT,HETELHHD,HETELAVL,HEPHONEO,...,PECERT1,PECERT2,PECERT3,PXCERT1,PXCERT2,PXCERT3,ID,ID_PERRP,HH_ID,IND_ID
0,4795110719,1,2020,1,201,-1,1,1,-1,1,...,2,-1,-1,20,0,0,47951107199011,4795110719901140,47951107199011,4795110719901140
1,4795110719,1,2020,1,201,-1,1,1,-1,1,...,2,-1,-1,20,0,0,47951107199011,4795110719901142,47951107199011,4795110719901142
2,4795110719,1,2020,3,201,-1,1,1,-1,1,...,2,-1,-1,20,0,0,479511071910011,47951107191001140,479511071910011,47951107191001140
3,4795110719,1,2020,3,201,-1,1,1,-1,1,...,2,-1,-1,20,0,0,479511071910011,47951107191001148,479511071910011,47951107191001148
4,4795110719,1,2020,3,201,-1,1,1,-1,1,...,2,-1,-1,20,0,0,479511071910011,47951107191001155,479511071910011,47951107191001155


Although initially `IND_ID` could have been a unique identifer, it seems that even in 

In [46]:
print(jan_2020_df.shape)
jan_2020_df.IND_ID.duplicated().value_counts()

(138697, 391)


False    121907
True      16790
Name: IND_ID, dtype: int64

`IND_ID` = 5440319005164911148 is an individual ID that is repated three times.

In [55]:
jan_2020_df[jan_2020_df.IND_ID.duplicated() == True]

,HRHHID,HRMONTH,HRYEAR4,HURESPLI,HUFINAL,HETENURE,HEHOUSUT,HETELHHD,HETELAVL,HEPHONEO,...,PECERT1,PECERT2,PECERT3,PXCERT1,PXCERT2,PXCERT3,ID,ID_PERRP,HH_ID,IND_ID
34,110665564993,1,2020,1,201,-1,1,1,-1,1,...,-1,-1,-1,-1,-1,-1,11066556499310011,1106655649931001148,11066556499310011,1106655649931001148
56,5440319005164,1,2020,2,201,-1,1,1,-1,1,...,-1,-1,-1,-1,-1,-1,54403190051649111,5440319005164911148,54403190051649111,5440319005164911148
57,5440319005164,1,2020,2,201,-1,1,1,-1,1,...,-1,-1,-1,-1,-1,-1,54403190051649111,5440319005164911148,54403190051649111,5440319005164911148
58,5440319005164,1,2020,2,201,-1,1,1,-1,1,...,-1,-1,-1,-1,-1,-1,54403190051649111,5440319005164911148,54403190051649111,5440319005164911148
65,5660614093159,1,2020,1,201,-1,1,1,-1,1,...,2,-1,-1,20,0,0,56606140931599011,5660614093159901151,56606140931599011,5660614093159901151
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
138638,802505343610855,1,2020,1,201,-1,1,1,-1,1,...,-1,-1,-1,-1,-1,-1,80250534361085511111,8025053436108551111148,80250534361085511111,8025053436108551111148
138645,802505349610555,1,2020,1,201,-1,1,1,-1,0,...,-1,-1,-1,-1,-1,-1,8025053496105559011,802505349610555901148,8025053496105559011,802505349610555901148
138646,802505349610555,1,2020,1,201,-1,1,1,-1,0,...,2,-1,-1,0,0,0,8025053496105559011,802505349610555901148,8025053496105559011,802505349610555901148
138670,905517039610115,1,2020,2,201,-1,1,1,-1,1,...,-1,-1,-1,-1,-1,-1,9055170396101159012,905517039610115901248,9055170396101159012,905517039610115901248


The last two digits of `IND_ID` shows the relationship to the reference person. Since thea above `IND_ID` had 48 at the end in three different duplicates of it, then it must be the case that they must all have the same relationship to the reference person. 

In [58]:
jan_2020_df[jan_2020_df.IND_ID.duplicated() == True].PERRP.value_counts()

48    14109
49      708
52      511
50      414
55      382
51      262
58      136
54       79
57       69
59       68
53       39
56       13
Name: PERRP, dtype: int64

48	CHILD
49	GRANDCHILD
50	PARENT
51	BROTHER/SISTER
52	OTHER REL. OR REF. PERSON
53	FOSTER CHILD
54	HOUSEMATE/ROOMMATE W/RELS.
55	HOUSEMATE/ROOMMATEW/O RELS.
56	ROOMER/BOARDER W/RELS.
57	ROOMER/BOARDER W/OUT RELS.
58	NONRELATIVE OF REFERENCE PERSON W/RELS.
59	NONRELATIVE OF REFERENCE PERSON W/OUT RELS.

Finding out the age for each of the three IND_ID duplicates shows that they are different people. Adding Sex and Age should fix this issue. Will be done in the .py file and executed in cells above.

In [64]:
jan_2020_df[(jan_2020_df.IND_ID.duplicated() == True) & (jan_2020_df.IND_ID == '5440319005164911148')].PRTAGE

56     4
57     8
58    12
Name: PRTAGE, dtype: int64

Adding sex and age to the individual ID helps a a great bit, but there are still a number of duplicates

In [66]:
print(jan_2020_df.shape)
jan_2020_df.IND_ID_SEX_AGE.duplicated().value_counts()

(138697, 392)


False    138020
True        677
Name: IND_ID_SEX_AGE, dtype: int64

You still get duplicates, most likely explained by twins or other similarities. `PULINENO` apparently will help provide unique IDs. https://www2.census.gov/programs-surveys/cps/methodology/How%20To%20Link%20CPS%20Public%20Use%20Files.pdf 

In [70]:
jan_2020_df[jan_2020_df.IND_ID_SEX_AGE.duplicated() == True].IND_ID_SEX_AGE.value_counts()

1867510102909711101155120    7
503105806229721001155120     5
208840315064488901159285     3
1903140100441361101157253    3
151146401400209901255120     3
                            ..
15050017074949100114815      1
179794090101846901148121     1
8025022034103961101148115    1
9939601007008031101148115    1
3023006719108421001148216    1
Name: IND_ID_SEX_AGE, Length: 624, dtype: int64

In [76]:
jan_2020_df.groupby('IND_ID_FINAL').filter(lambda x: len(x) > 1)

,HRHHID,HRMONTH,HRYEAR4,HURESPLI,HUFINAL,HETENURE,HEHOUSUT,HETELHHD,HETELAVL,HEPHONEO,...,PECERT3,PXCERT1,PXCERT2,PXCERT3,ID,ID_PERRP,HH_ID,IND_ID,IND_ID_SEX_AGE,IND_ID_FINAL


No more duplicates

In [79]:
jan_2020_df.IND_ID_FINAL.duplicated().value_counts()

False    138697
Name: IND_ID_FINAL, dtype: int64

## Determining churn

### Jan to Feb

In [99]:
churn_value_counts = jan_feb_2020_df.IND_ID_FINAL.duplicated().value_counts()
churn_value_counts

False    190093
True      87852
Name: IND_ID_FINAL, dtype: int64

In [97]:
non_duplicates = churn_value_counts.values[0]
duplicates = churn_value_counts.values[1]

retention = duplicates / (duplicates + non_duplicates)
retention

0.3160769216931407

In [86]:
jan_feb_2020_df[(jan_feb_2020_df.IND_ID_FINAL.duplicated() == True)]

,HRHHID,HRMONTH,HRYEAR4,HURESPLI,HUFINAL,HETENURE,HEHOUSUT,HETELHHD,HETELAVL,HEPHONEO,...,PECERT3,PXCERT1,PXCERT2,PXCERT3,ID,ID_PERRP,HH_ID,IND_ID,IND_ID_SEX_AGE,IND_ID_FINAL
0,4795110719,2,2020,1,201,-1,1,1,-1,1,...,-1,20,0,0,47951107199011,4795110719901140,47951107199011,4795110719901140,4795110719901140169,47951107199011401691
1,4795110719,2,2020,1,201,-1,1,1,-1,1,...,-1,20,0,0,47951107199011,4795110719901142,47951107199011,4795110719901142,4795110719901142265,47951107199011422652
2,4795110719,2,2020,1,201,-1,1,1,-1,1,...,-1,20,0,0,479511071910011,47951107191001140,479511071910011,47951107191001140,47951107191001140154,479511071910011401541
4,4795110719,2,2020,1,201,-1,1,1,-1,1,...,-1,20,0,0,479511071910011,47951107191001155,479511071910011,47951107191001155,47951107191001155248,479511071910011552482
6,71491002941,2,2020,1,201,-1,1,1,-1,1,...,-1,20,0,0,7149100294111111,714910029411111142,7149100294111111,714910029411111142,714910029411111142238,7149100294111111422382
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
139235,926363801781501,2,2020,1,201,-1,1,1,-1,1,...,-1,20,0,0,9263638017815019111,926363801781501911140,9263638017815019111,926363801781501911140,926363801781501911140165,9263638017815019111401651
139236,926363801781501,2,2020,1,201,-1,1,1,-1,1,...,-1,20,0,0,9263638017815019111,926363801781501911142,9263638017815019111,926363801781501911142,926363801781501911142262,9263638017815019111422622
139238,956729301591507,2,2020,1,201,-1,1,1,-1,0,...,-1,20,0,0,9567293015915079011,956729301591507901142,9567293015915079011,956729301591507901142,956729301591507901142270,9567293015915079011422702
139241,965568100700708,2,2020,1,223,-1,1,1,-1,1,...,-1,-1,-1,-1,96556810070070811111,96556810070070811111-1,96556810070070811111,96556810070070811111-1,96556810070070811111-1-1-1,96556810070070811111-1-1-1-1


In [87]:
jan_feb_2020_df[(jan_feb_2020_df.IND_ID_FINAL == '47951107199011401691')]

,HRHHID,HRMONTH,HRYEAR4,HURESPLI,HUFINAL,HETENURE,HEHOUSUT,HETELHHD,HETELAVL,HEPHONEO,...,PECERT3,PXCERT1,PXCERT2,PXCERT3,ID,ID_PERRP,HH_ID,IND_ID,IND_ID_SEX_AGE,IND_ID_FINAL
0,4795110719,1,2020,1,201,-1,1,1,-1,1,...,-1,20,0,0,47951107199011,4795110719901140,47951107199011,4795110719901140,4795110719901140169,47951107199011401691
0,4795110719,2,2020,1,201,-1,1,1,-1,1,...,-1,20,0,0,47951107199011,4795110719901140,47951107199011,4795110719901140,4795110719901140169,47951107199011401691


### Jan to Apr

In [100]:
jan_apr_2020_df = dl.CPS_raw(targetdir, ['jan2020', 'apr2020'], None)
jan_apr_2020_df = c.clean_CPS_df(jan_apr_2020_df)

churn_value_counts = jan_apr_2020_df.IND_ID_FINAL.duplicated().value_counts()
churn_value_counts

False    245999
True      22080
Name: IND_ID_FINAL, dtype: int64

In [101]:
non_duplicates = churn_value_counts.values[0]
duplicates = churn_value_counts.values[1]

retention = duplicates / (duplicates + non_duplicates)
retention

0.08236378082580136